<a href="https://colab.research.google.com/github/Praveen76/RAG-Retrieval-Augmented-Generation-using-Langchain/blob/main/RAG_using_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



### **Installing and importing packages**

In [1]:
!pip install openai
!pip install langchain
!pip install langchain-openai langchain_community
!pip install pypdf
!pip install chromadb
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.9 MB/s eta 0:00:00


In [2]:
import os
import openai
import numpy as np

#### **Authentication for OpenAI API**

In [3]:
import yaml
import json

from google.colab import drive

# Mount Google Drive with force remount
drive.mount('/content/drive', force_remount=True)

# Load relevant API Keys
file_path = '/content/drive/MyDrive/.API_KEYS/API_KEYS.yml'

with open(file_path, 'r') as file:
    api_keys = yaml.safe_load(file)



Mounted at /content/drive


In [9]:


# Extract openai username and key
openai_key = api_keys['OPEN_AI']['Key']

os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key= os.getenv('OPENAI_API_KEY')


# Extract Kaggle username and key
aws_access_key_id = api_keys['AWS']['AWS_ACCESS_KEY_ID']
aws_secret_access_key = api_keys['AWS']['AWS_SECRET_ACCESS_KEY']

os.environ['AWS_ACCESS_KEY_ID']=aws_access_key_id
os.environ['AWS_SECRET_ACCESS_KEY']=aws_secret_access_key
del aws_access_key_id, aws_secret_access_key


In [13]:
!pip install awscli
!aws s3 cp s3://datasciencedocx/RAG_using_Langchain/ . --recursive


download: s3://datasciencedocx/RAG_using_Langchain/ens_d2.pdf to ./ens_d2.pdf
download: s3://datasciencedocx/RAG_using_Langchain/pca_d1.pdf to ./pca_d1.pdf
download: s3://datasciencedocx/RAG_using_Langchain/LangChain.pdf to ./LangChain.pdf


### **Loading the documents**

[PDF Loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)

In [14]:
from langchain_community.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose
    PyPDFLoader("/content/pca_d1.pdf"),
    PyPDFLoader("/content/ens_d2.pdf"),
    PyPDFLoader("/content/ens_d2.pdf"),
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [15]:
print(docs[0].page_content)

 
1  
 N  
1 Principal  Component  Analysis  
In real world data analysis tasks we analyze complex data i.e. multi dimensional data. We plot the  
data and find various patterns in it or use it to train some machine learning models.  One way to  
think  about  dimensions  is that  suppose  you have  an data  point  x , if we consider  this data  point  as 
a physical  object  then  dimensions  are merely  a basis  of view,  like where  is the data  located  when  
it is observed  from  horizontal  axis or vertical  axis.  
As the dimensions  of data  increases,  the difficulty  to visualize  it and perform  computations  on 
it also increases.  So, how  to reduce  the dimensions  of a data: - 
• Remove  the redundant  dimensions  
• Only keep the most important dimensions  
Let us first try to understand  some  terms: - 
Variance : It is a measure of the variability or it simply measures how spread the data set is.  
Mathematically,  it is the average  squared  deviation  from  the mea

### **Splitting of document**

[Recursively split by character](https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter)

[Split by character](https://python.langchain.com/docs/modules/data_connection/document_transformers/character_text_splitter)


In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [17]:
# Split
#from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

In [18]:
splits = text_splitter.split_documents(docs)
print(len(splits))
splits

27


[Document(page_content='1  \n N  \n1 Principal  Component  Analysis  \nIn real world data analysis tasks we analyze complex data i.e. multi dimensional data. We plot the  \ndata and find various patterns in it or use it to train some machine learning models.  One way to  \nthink  about  dimensions  is that  suppose  you have  an data  point  x , if we consider  this data  point  as \na physical  object  then  dimensions  are merely  a basis  of view,  like where  is the data  located  when', metadata={'source': '/content/pca_d1.pdf', 'page': 0}),
 Document(page_content='it is observed  from  horizontal  axis or vertical  axis.  \nAs the dimensions  of data  increases,  the difficulty  to visualize  it and perform  computations  on \nit also increases.  So, how  to reduce  the dimensions  of a data: - \n• Remove  the redundant  dimensions  \n• Only keep the most important dimensions  \nLet us first try to understand  some  terms: - \nVariance : It is a measure of the variability or it s

### **Embeddings**

Let's take our splits and embed them.

In [19]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [20]:
embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7904a0341630>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7904a0377520>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

### **Understanding similarity search with a toy example**

In [21]:
sentence1 = "i like dogs"
sentence2 = "i like cats"
sentence3 = "the weather is ugly, too hot outside"

In [22]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [23]:
len(embedding1), len(embedding2), len(embedding3)

(1536, 1536, 1536)

In [24]:
# Check 3///;[[]] similarity45
np.dot(embedding1, embedding2), np.dot(embedding1, embedding3),np.dot(embedding2, embedding3)

(0.9200098139975112, 0.7636469209069651, 0.758339267629513)

### **Vectorstores**

In [25]:
from langchain_community.vectorstores import Chroma # Light-weight and in memory

In [26]:
persist_directory = 'docs/chroma/'
!rm -rf ./docs/chroma  # remove old database files if any

In [27]:
vectordb = Chroma.from_documents(
    documents=splits, # splits we created earlier
    embedding=embedding,
    persist_directory=persist_directory # save the directory
)

In [28]:
vectordb.persist() # Let's **save vectordb** so we can use it later!

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [29]:
print(vectordb._collection.count()) # same as number of splites

27


### **Similarity Search**

In [30]:
question = "how does pca reduce the dimension?"

In [31]:
docs = vectordb.similarity_search(question,k=3) # k --> No. of doc as return
print(len(docs))
print(docs[0].page_content)
print(docs[1].page_content)
print(docs[2].page_content)

3
This defines  the goal  of PCA: - 
1. Find  linearly  independent  dimensions  which  can losslessly  represent  the data  points.  
2. Those  newly  found  dimensions  should  allow  us to predict/reconstruct  the original  dimensions.
2  
  
So, what does  Principal  Component  Analysis  (PCA)  do?  
PCA finds a new set of dimensions (or a set of basis of views) such that all the dimensions are  
orthogonal (and hence linearly independent) and ranked according to the variance of data along  
them. It means more important principle axis occurs first. (more important = more variance/more  
spread  out data)  
 
How  does  PCA  work?  
• Calculate  the covariance matrix  X of data  points.
1  
 N  
1 Principal  Component  Analysis  
In real world data analysis tasks we analyze complex data i.e. multi dimensional data. We plot the  
data and find various patterns in it or use it to train some machine learning models.  One way to  
think  about  dimensions  is that  suppose  you have  a

### **Edge case where failure may happen**

1. Lack of Diversity : Semantic search fetches all similar documents, but does not enforce diversity.

    - Notice that we're getting duplicate chunks (because of the duplicate `ens_d2.pdf` in the index). `docs[0]` and `docs[1]` are indentical.

  **Addressing Diversity - MMR-Maximum Marginal Relevance**

2. Lack of spefificity:  The question may be from a particular doc but answer may contain information from other doc.

  **Addressing Specificity: Working with metadata - Manually**

  **Working with metadata using self-query retriever -Automatically**

  **Example 1. Addressing Diversity - MMR-Maximum Marginal Relevance**

In [38]:
question= 'how ensemble method works?'
docs = vectordb.similarity_search(question,k=5) # Without MMR

In [39]:
docs[0]

Document(page_content='1   \nEnsemble  Methods  \nLet us consider  a real  world  situation  which  uses  Ensemble  Methods,  which  is, when  a user  wants  \nto buy a new product. Many users who have already purchased that product will have given either  \npositive  or negative  ratings.  If in the group,  many  users  have  given  positive  ratings,  then  the \ncombined  rating  will be positive.  Instead  of a single  rating,  the ratings  of the group  of users  is', metadata={'page': 0, 'source': '/content/ens_d2.pdf'})

In [40]:
docs[1]

Document(page_content='1   \nEnsemble  Methods  \nLet us consider  a real  world  situation  which  uses  Ensemble  Methods,  which  is, when  a user  wants  \nto buy a new product. Many users who have already purchased that product will have given either  \npositive  or negative  ratings.  If in the group,  many  users  have  given  positive  ratings,  then  the \ncombined  rating  will be positive.  Instead  of a single  rating,  the ratings  of the group  of users  is', metadata={'page': 0, 'source': '/content/ens_d2.pdf'})

In [41]:
docs[2]

Document(page_content='considered.  The product  is bought  by the user  when  the combined  ratings  of the group  is positive.  \nThe user  gets  a fairer  idea  about  the product  when  all the ratings  are combined.  \nHere, the combination of ratings is done so that the decision making process of the user is made  \neasy.  \nEnsemble Methods refer to combining many different machine learning models in order to get a  \nmore  powerful  prediction.', metadata={'page': 0, 'source': '/content/ens_d2.pdf'})

In [42]:
docs[3]

Document(page_content='considered.  The product  is bought  by the user  when  the combined  ratings  of the group  is positive.  \nThe user  gets  a fairer  idea  about  the product  when  all the ratings  are combined.  \nHere, the combination of ratings is done so that the decision making process of the user is made  \neasy.  \nEnsemble Methods refer to combining many different machine learning models in order to get a  \nmore  powerful  prediction.', metadata={'page': 0, 'source': '/content/ens_d2.pdf'})

Addressing Diversity - MMR-Maximum Marginal Relevance

In [43]:
docs_with_mmr=vectordb.max_marginal_relevance_search(question, k=3, fetch_k=6) # With MMR

In [44]:
docs_with_mmr[0]

Document(page_content='1   \nEnsemble  Methods  \nLet us consider  a real  world  situation  which  uses  Ensemble  Methods,  which  is, when  a user  wants  \nto buy a new product. Many users who have already purchased that product will have given either  \npositive  or negative  ratings.  If in the group,  many  users  have  given  positive  ratings,  then  the \ncombined  rating  will be positive.  Instead  of a single  rating,  the ratings  of the group  of users  is', metadata={'page': 0, 'source': '/content/ens_d2.pdf'})

In [45]:
docs_with_mmr[1]

Document(page_content='considered.  The product  is bought  by the user  when  the combined  ratings  of the group  is positive.  \nThe user  gets  a fairer  idea  about  the product  when  all the ratings  are combined.  \nHere, the combination of ratings is done so that the decision making process of the user is made  \neasy.  \nEnsemble Methods refer to combining many different machine learning models in order to get a  \nmore  powerful  prediction.', metadata={'page': 0, 'source': '/content/ens_d2.pdf'})

In [46]:
docs_with_mmr[2]

Document(page_content='more  powerful  prediction.  \nThus,  ensemble  methods  increase  the accuracy  of the predictions.  \n \nWhy  use Ensemble  Methods?  \nEnsemble  Methods  are used  in order  to: \n• decrease  variance  (bagging)  \n• decrease  bias (boosting)  \n• improve  predictions  (stacking)  \n \nBagging  \nBagging  actually  refers  to Bootstrap  Aggregators.  \nBagging tests multiple models on the data by sampling and replacing data i.e it utilizes bootstrap -', metadata={'page': 0, 'source': '/content/ens_d2.pdf'})

 **Example 2. Addressing Specificity: Working with metadata - Manually**

In [47]:
# Without metadata information
question = "what is the role of variance in pca?"
docs = vectordb.similarity_search(question,k=5)
for doc in docs:
    print(doc.metadata) # metadata contains information about from which doc the answer has been fetched

{'page': 1, 'source': '/content/pca_d1.pdf'}
{'page': 2, 'source': '/content/pca_d1.pdf'}
{'page': 2, 'source': '/content/pca_d1.pdf'}
{'page': 0, 'source': '/content/pca_d1.pdf'}
{'page': 0, 'source': '/content/ens_d2.pdf'}


Notice above, the last information is from 'ens_d2' doc.

In [48]:
# With metadata information
question = "what is the role of variance in pca?"
docs = vectordb.similarity_search(
    question,
    k=5,
    filter={"source":'/content/pca_d1.pdf'} # manually passing metadata, using metadata filter.
)

for doc in docs:
    print(doc.metadata)

{'page': 1, 'source': '/content/pca_d1.pdf'}
{'page': 2, 'source': '/content/pca_d1.pdf'}
{'page': 2, 'source': '/content/pca_d1.pdf'}
{'page': 0, 'source': '/content/pca_d1.pdf'}
{'page': 0, 'source': '/content/pca_d1.pdf'}


[**Addressing Specificity -Automatically: Working with metadata using self-query retriever**](https://python.langchain.com/docs/modules/data_connection/retrievers/self_query)



### **Additional tricks: Compression**

Another approach for improving the quality of retrieved docs is compression. Information most relevant to a query may be buried in a document with a lot of irrelevant text. Passing that full document through your application can lead to more expensive LLM calls and poorer responses.

[Contextual compression](https://python.langchain.com/docs/modules/data_connection/retrievers/contextual_compression) is meant to fix this.

### **Retrieval + Question Answering :  Connecting with LLMs**

In [49]:
llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [57]:
question = "What is principal component analysis?"
docs = vectordb.max_marginal_relevance_search(question, k=2, fetch_k=5)
len(docs)

2

In [58]:
docs[0]

Document(page_content='2  \n  \nSo, what does  Principal  Component  Analysis  (PCA)  do?  \nPCA finds a new set of dimensions (or a set of basis of views) such that all the dimensions are  \northogonal (and hence linearly independent) and ranked according to the variance of data along  \nthem. It means more important principle axis occurs first. (more important = more variance/more  \nspread  out data)  \n \nHow  does  PCA  work?  \n• Calculate  the covariance matrix  X of data  points.', metadata={'page': 1, 'source': '/content/pca_d1.pdf'})

In [59]:
docs[1]

Document(page_content='1  \n N  \n1 Principal  Component  Analysis  \nIn real world data analysis tasks we analyze complex data i.e. multi dimensional data. We plot the  \ndata and find various patterns in it or use it to train some machine learning models.  One way to  \nthink  about  dimensions  is that  suppose  you have  an data  point  x , if we consider  this data  point  as \na physical  object  then  dimensions  are merely  a basis  of view,  like where  is the data  located  when', metadata={'page': 0, 'source': '/content/pca_d1.pdf'})

####**[RetrievalQA chain](https://docs.smith.langchain.com/cookbook/hub-examples/retrieval-qa-chain)**

####**[Vector store-backed retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/vectorstore)**

In [61]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

In [62]:
from langchain.chains import RetrievalQA

In [63]:
question = "What is principal component analysis?"

qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectordb.as_retriever(), return_source_documents=True)

result = qa_chain.invoke({"query": question})

In [64]:
result["result"]

'Principal Component Analysis (PCA) is a technique used in data analysis to find a new set of dimensions that are orthogonal (linearly independent) and ranked according to the variance of data along them. The goal of PCA is to find linearly independent dimensions that can represent the data points losslessly and allow for the prediction or reconstruction of the original dimensions. PCA works by calculating the covariance matrix of data points, finding eigenvectors and corresponding eigenvalues, sorting the eigenvectors based on eigenvalues, choosing the top k eigenvectors as new dimensions, and transforming the original n-dimensional data points into k dimensions. In simpler terms, PCA helps in reducing the dimensionality of data while preserving the most important information or patterns in the data.'

In [65]:
result["source_documents"]

[Document(page_content='2  \n  \nSo, what does  Principal  Component  Analysis  (PCA)  do?  \nPCA finds a new set of dimensions (or a set of basis of views) such that all the dimensions are  \northogonal (and hence linearly independent) and ranked according to the variance of data along  \nthem. It means more important principle axis occurs first. (more important = more variance/more  \nspread  out data)  \n \nHow  does  PCA  work?  \n• Calculate  the covariance matrix  X of data  points.', metadata={'page': 1, 'source': '/content/pca_d1.pdf'}),
 Document(page_content='1  \n N  \n1 Principal  Component  Analysis  \nIn real world data analysis tasks we analyze complex data i.e. multi dimensional data. We plot the  \ndata and find various patterns in it or use it to train some machine learning models.  One way to  \nthink  about  dimensions  is that  suppose  you have  an data  point  x , if we consider  this data  point  as \na physical  object  then  dimensions  are merely  a basis  of

###**Under the hood? --> Understanding RAG Prompt**

In [66]:
!pip install langchainhub

In [67]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

Use three sentences maximum.Keep the answer as concise as possible.

In [68]:
# Build prompt
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

In [69]:
QA_CHAIN_PROMPT

PromptTemplate(input_variables=['context', 'question'], template='Use the following pieces of context to answer the question at the end.\nIf you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\nAlways say "thanks for asking!" at the end of the answer.\n{context}\nQuestion: {question}\nHelpful Answer:')

In [70]:
# Run chain
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(search_type="mmr",search_kwargs={"k": 2, "fetch_k":6} ), # "k":2, "fetch_k":3
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
                                       return_source_documents=True
                                       )

In [71]:
qa_chain

RetrievalQA(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template='Use the following pieces of context to answer the question at the end.\nIf you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\nAlways say "thanks for asking!" at the end of the answer.\n{context}\nQuestion: {question}\nHelpful Answer:'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x79049c6fc970>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x79049c6fece0>, temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='')), document_variable_name='context'), return_source_documents=True, retriever=VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7904a04e9060>, search_type='mmr', search_kwargs={'k': 2, 'fetch_k': 6}))

**Example 1**

In [72]:
question = "What is principal component analysis?"
result = qa_chain.invoke({"query": question})
result["source_documents"]

[Document(page_content='2  \n  \nSo, what does  Principal  Component  Analysis  (PCA)  do?  \nPCA finds a new set of dimensions (or a set of basis of views) such that all the dimensions are  \northogonal (and hence linearly independent) and ranked according to the variance of data along  \nthem. It means more important principle axis occurs first. (more important = more variance/more  \nspread  out data)  \n \nHow  does  PCA  work?  \n• Calculate  the covariance matrix  X of data  points.', metadata={'page': 1, 'source': '/content/pca_d1.pdf'}),
 Document(page_content='1  \n N  \n1 Principal  Component  Analysis  \nIn real world data analysis tasks we analyze complex data i.e. multi dimensional data. We plot the  \ndata and find various patterns in it or use it to train some machine learning models.  One way to  \nthink  about  dimensions  is that  suppose  you have  an data  point  x , if we consider  this data  point  as \na physical  object  then  dimensions  are merely  a basis  of

In [73]:
result["result"]

'Principal Component Analysis (PCA) is a technique used to find a new set of dimensions that are orthogonal and ranked according to the variance of data along them. The more important principle axis occurs first, based on the variance or spread out data. Thanks for asking!'

**Example 2**

In [74]:
question = "What does it say about variance in context of both PCA and Ensemble?"
result = qa_chain({"query": question})
result["source_documents"]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='2  \n  \nSo, what does  Principal  Component  Analysis  (PCA)  do?  \nPCA finds a new set of dimensions (or a set of basis of views) such that all the dimensions are  \northogonal (and hence linearly independent) and ranked according to the variance of data along  \nthem. It means more important principle axis occurs first. (more important = more variance/more  \nspread  out data)  \n \nHow  does  PCA  work?  \n• Calculate  the covariance matrix  X of data  points.', metadata={'page': 1, 'source': '/content/pca_d1.pdf'}),
 Document(page_content='more  powerful  prediction.  \nThus,  ensemble  methods  increase  the accuracy  of the predictions.  \n \nWhy  use Ensemble  Methods?  \nEnsemble  Methods  are used  in order  to: \n• decrease  variance  (bagging)  \n• decrease  bias (boosting)  \n• improve  predictions  (stacking)  \n \nBagging  \nBagging  actually  refers  to Bootstrap  Aggregators.  \nBagging tests multiple models on the data by sampling and replacin

In [75]:
result["result"]

'Both PCA and Ensemble methods, such as Bagging, aim to decrease variance. PCA does this by finding a new set of dimensions that are orthogonal and ranked according to the variance of the data, while Ensemble methods like Bagging decrease variance by testing multiple models on the data through sampling and replacing data. Thanks for asking!'

### **RetrievalQA chain types : [Map reduce, Refine, Map rerank(Legacy)](https://python.langchain.com/docs/modules/chains/)**

- Whatever techniques we havae used is stuff method (default - chain_type="stuff")and there is only one call to LLM

In [76]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_type="mmr",search_kwargs={"k": 4, "fetch_k":8}),
    chain_type="map_reduce"
)

In [77]:
question ="What principal component analysis?"
result = qa_chain_mr({"query": question})
result["result"]

'Principal Component Analysis (PCA) is a statistical technique used to simplify and reduce the dimensionality of data by finding a new set of dimensions that are orthogonal (linearly independent) and ranked according to the variance of the data along them. PCA helps in identifying the most important axes or components that capture the most variation in the data. It is commonly used in data analysis to reduce complexity, visualize data, identify patterns, and extract important features in various fields such as machine learning, image processing, and finance.'

### **Make it like Chatbot : Adding Memory**

In [78]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [79]:
# Run chain
from langchain.chains import ConversationalRetrievalChain
qa= ConversationalRetrievalChain.from_llm(llm,
                                       retriever=vectordb.as_retriever(search_type="mmr",search_kwargs={"k": 4, "fetch_k":8} ), # "k":2, "fetch_k":3
                                       memory=memory
                                       )

In [80]:
question = "tell me something about PCA"
result = qa.invoke({"question": question})

In [81]:
result['answer']

'Principal Component Analysis (PCA) is a technique used in data analysis and machine learning to simplify the complexity of high-dimensional data by transforming it into a new set of dimensions called principal components. These components are orthogonal (linearly independent) and are ranked based on the variance of the data along them. The more important principal axes, which capture the most variance or spread out the data, occur first.\n\nPCA works by first calculating the covariance matrix of the data points. The main goals of PCA are to find linearly independent dimensions that can represent the data losslessly and to allow the prediction or reconstruction of the original dimensions. This is achieved by calculating eigenvectors and corresponding eigenvalues, sorting the eigenvectors based on their eigenvalues in decreasing order, choosing the first k eigenvectors as the new k dimensions, and transforming the original n-dimensional data points into k dimensions.\n\nEigenvectors rep

In [82]:
question = "please list point-wise,  how does pca works?"
result = qa({"question": question})

In [83]:
print(result['answer'])

Principal Component Analysis (PCA) works in the following steps:

1. Calculate the covariance matrix X of data points.
2. Find linearly independent dimensions that can losslessly represent the data points.
3. Calculate eigenvectors and corresponding eigenvalues.
4. Sort the eigenvectors according to their eigenvalues in decreasing order.
5. Choose the first k eigenvectors, which will be the new k dimensions.
6. Transform the original n-dimensional data points into k dimensions.

It is important to understand eigenvalues and eigenvectors in order to grasp the detailed working of PCA. Eigenvectors represent the directions in which our data are dispersed. By transforming the original data points, the goal is to achieve a diagonal covariance matrix. It is recommended to normalize the data before performing PCA to avoid misleading components, especially when using features of different scales. Additionally, using a correlation matrix instead of a covariance matrix can be beneficial when dea

In [84]:
question = "what do we get from covariance matrix for doing PCA?"
result = qa({"question": question})
print(result['answer'])

When performing Principal Component Analysis (PCA), the covariance matrix provides information about the relationships between the different dimensions or features in the dataset. Specifically, the covariance matrix helps in identifying how the dimensions are related to each other in terms of variance and covariance. By analyzing the covariance matrix, PCA determines the directions in which the data varies the most, which are represented by the eigenvectors and eigenvalues of the covariance matrix. The eigenvectors represent the principal components (new set of dimensions) that are orthogonal and ranked based on the variance of the data along them, while the eigenvalues indicate the relative importance of these different directions. In summary, the covariance matrix in PCA helps in understanding the structure and variability of the data in order to find the most important dimensions for dimensionality reduction.


### **Download the vector DB**

In [85]:
# Zip the entire folder
!zip -r /content/docs.zip /content/docs

  adding: content/docs/ (stored 0%)
  adding: content/docs/chroma/ (stored 0%)
  adding: content/docs/chroma/chroma.sqlite3 (deflated 58%)
  adding: content/docs/chroma/e6a68376-3554-4f31-99fa-7204090f366d/ (stored 0%)
  adding: content/docs/chroma/e6a68376-3554-4f31-99fa-7204090f366d/link_lists.bin (stored 0%)
  adding: content/docs/chroma/e6a68376-3554-4f31-99fa-7204090f366d/data_level0.bin (deflated 100%)
  adding: content/docs/chroma/e6a68376-3554-4f31-99fa-7204090f366d/header.bin (deflated 61%)
  adding: content/docs/chroma/e6a68376-3554-4f31-99fa-7204090f366d/length.bin (deflated 98%)


In [86]:
from google.colab import files
files.download("/content/docs.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### **Upload the vector db from previous step and unzip**

In [ ]:
!unzip /content/docs.zip  -d /

Archive:  /content/docs.zip
replace /content/docs/chroma/chroma.sqlite3? [y]es, [n]o, [A]ll, [N]one, [r]ename: 